In [1]:
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import GridSearchCV
#from keras.wrappers.scikit_learn import KerasClassifier
#from keras.optimizers import SGD
#from keras.constraints import maxnorm
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.callbacks
import tensorflow.keras.datasets.fashion_mnist

In [2]:
(x_trg_data,y_trg_data),(x_test_data,y_test_data) = tensorflow.keras.datasets.fashion_mnist.load_data()

In [3]:
print(x_trg_data.shape)
print(y_trg_data.shape)
print(x_test_data.shape)
print(y_test_data.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [4]:
print(x_trg_data.shape[0],'train set samples')
print(y_trg_data.shape[0],'train set samples')

60000 train set samples
60000 train set samples


In [5]:
print(np.min(x_trg_data))
print(np.max(x_trg_data))

0
255


In [6]:
x_trg_data = x_trg_data.astype('float32')/255
x_test_data = x_test_data.astype('float32')/255
print(np.min(x_trg_data))
print(np.max(x_trg_data))
print(x_trg_data[:2])

0.0
1.0
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
np.random.seed(500)

In [13]:
x_trg,x_val,y_trg,y_val = train_test_split(x_trg_data,y_trg_data,test_size = 0.2,random_state = 500)

In [14]:
x_test,y_test = x_test_data,y_test_data

In [15]:
x_trg = x_trg.reshape(x_trg.shape[0],28,28,1)
x_val = x_val.reshape(x_val.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

In [16]:
y_trg = tf.keras.utils.to_categorical(y_trg,10)
y_val = tf.keras.utils.to_categorical(y_val,10)
y_test = tf.keras.utils.to_categorical(y_test,10)

In [17]:
print(x_trg.shape)
print(y_trg.shape)

(48000, 28, 28, 1)
(48000, 10)


In [18]:
print(x_trg.shape[0])
print(x_val.shape[0])
print(x_test.shape[0])

48000
12000
10000


In [19]:
model1 = tf.keras.Sequential()

In [20]:
model1.add(tf.keras.layers.Flatten(input_shape = (28,28,1)))
model1.add(tf.keras.layers.Dense(10,activation = 'softmax'))

In [21]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 10)                7850      
                                                                 
Total params: 7850 (30.66 KB)
Trainable params: 7850 (30.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model1.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [23]:
model1.fit(x_trg,y_trg,batch_size = 128,epochs = 3,validation_data = (x_val,y_val))

Epoch 1/3
375/375 [==============================] - 1s 3ms/step - loss: 0.7786 - accuracy: 0.7448 - val_loss: 0.5636 - val_accuracy: 0.8157
Epoch 2/3
375/375 [==============================] - 1s 2ms/step - loss: 0.5373 - accuracy: 0.8217 - val_loss: 0.4968 - val_accuracy: 0.8301
Epoch 3/3
375/375 [==============================] - 1s 2ms/step - loss: 0.4920 - accuracy: 0.8341 - val_loss: 0.4697 - val_accuracy: 0.8385


In [24]:
scores = model1.evaluate(x_test, y_test, verbose = 0)
print("Accuracy: ",100*scores[1])

Accuracy:  82.41000175476074


In [25]:
model2 = tf.keras.Sequential()

In [27]:
model2.add(tf.keras.layers.Conv2D(filters = 64,kernel_size = 2,padding = 'same',activation = 'relu',input_shape = (28,28,1)))
model2.add(tf.keras.layers.MaxPooling2D(pool_size = 2))
model2.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 2,padding = 'same',activation = 'relu'))
model2.add(tf.keras.layers.MaxPooling2D(pool_size = 2))
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(128,activation = 'relu'))
model2.add(tf.keras.layers.Dense(64,activation = 'relu'))
model2.add(tf.keras.layers.Dense(10,activation = 'softmax'))

In [28]:
model2.summary()
model2.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        8224      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 32)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 1568)              0         
                                                                 
 dense_1 (Dense)             (None, 128)              

In [29]:
model2.fit(x_trg,y_trg,batch_size = 256,epochs = 3,validation_data = (x_val,y_val))

Epoch 1/3
188/188 [==============================] - 34s 178ms/step - loss: 0.6824 - accuracy: 0.7536 - val_loss: 0.4114 - val_accuracy: 0.8537
Epoch 2/3
188/188 [==============================] - 33s 174ms/step - loss: 0.3896 - accuracy: 0.8595 - val_loss: 0.3861 - val_accuracy: 0.8584
Epoch 3/3
188/188 [==============================] - 33s 174ms/step - loss: 0.3415 - accuracy: 0.8771 - val_loss: 0.3027 - val_accuracy: 0.8923


In [30]:
test_score = model2.evaluate(x_test, y_test, verbose = 0)
print("Accuracy test_score: ",100*test_score[1])
train_score = model2.evaluate(x_trg, y_trg, verbose = 0)
print("Accuracy train_score: ",100*train_score[1])

Accuracy test_score:  87.91999816894531
Accuracy train_score:  89.31458592414856
